In [202]:
import pandas as pd
from datetime import datetime
import pytz

import yfinance as yf

from deltalake.writer import write_deltalake
from deltalake import DeltaTable

# Introduction
In my daily work, I frequently use Pandas. To better understand timezone-naive and timezone-aware concepts, I’ll focus on the following three tools:

- datetime.datetime: Part of Python's built-in datetime module, capable of handling up to microseconds.
- pd.Timestamp: Ideal for working with pandas DataFrames and time-series data.
- pd.to_datetime: Useful for converting date columns from files or databases into pandas-compatible datetime objects.

# Let's get started with the basics
## Naive Time

In [122]:
dt = datetime.now()
dt

datetime.datetime(2025, 1, 22, 23, 35, 25, 699761)

In [124]:
dt.isoformat()

'2025-01-22T23:35:25.699761'

In [126]:
pd_dt = pd.Timestamp(dt)
pd_dt

Timestamp('2025-01-22 23:35:25.699761')

In [128]:
pd_dt = pd.to_datetime(dt)
pd_dt

Timestamp('2025-01-22 23:35:25.699761')

## Aware Time

In [112]:
# Current time, timezone aware
dt = datetime.now(pytz.utc)
dt

datetime.datetime(2025, 1, 22, 23, 34, 18, 759984, tzinfo=<UTC>)

In [114]:
dt.isoformat()

'2025-01-22T23:34:18.759984+00:00'

In [82]:
pd.Timestamp(dt)

Timestamp('2025-01-22 23:30:03.029561+0000', tz='UTC')

In [84]:
pd.to_datetime(dt)

Timestamp('2025-01-22 23:30:03.029561+0000', tz='UTC')

### Let's get started with basics pandas DF

In [195]:
df = get_prices()
df["dt_aware"] = datetime.now(pytz.utc)
df["dt_naive"] = datetime.now()

[*********************100%***********************]  1 of 1 completed


In [197]:
df.dtypes

date             datetime64[ns]
ticker                   object
close                   float64
high                    float64
low                     float64
open                    float64
volume                    int64
dt_aware    datetime64[us, UTC]
dt_naive         datetime64[us]
dtype: object

In [209]:
df.head()

,date,ticker,close,high,low,open,volume,dt_aware,dt_naive
0,2025-01-02,AAPL,243.850006,249.100006,241.820007,248.929993,55740700,2025-01-23 00:02:40.462365+00:00,2025-01-23 00:02:40.462847
1,2025-01-03,AAPL,243.360001,244.179993,241.889999,243.360001,40244100,2025-01-23 00:02:40.462365+00:00,2025-01-23 00:02:40.462847


In [177]:
def get_prices():
    tickers = ['AAPL']
    data = yf.download(tickers, start='2025-01-01', end='2025-01-05')
    
    df = data.stack(future_stack=True).reset_index()
    df.columns = [i for i in df.columns.values]
    df.columns = [col.lower() for col in df.columns]
    return df

In [189]:
delta_directory = os.path.join("tmp", "timezone", "price")

In [200]:
import time

executions = [1, 2, 3, 4]

for exeuctions in executions:
    df = get_prices()
    df["dt_aware"] = datetime.now(pytz.utc)
    df["dt_naive"] = datetime.now()
    
    write_deltalake(
        table_or_uri=delta_directory,
        data=df,
        mode="append",
        engine="rust",
        )
    time.sleep(30)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [203]:
dl = DeltaTable(delta_directory)

### Let's try filtering on the date column

In [219]:
#string filters
try:
    filters = [("date", "==", "2025-01-02")]
    df_filtered = dl.to_pandas(filters=filters)
except Exception as e:
    print(e)

Function 'equal' has no kernel matching input types (timestamp[us], string)


In [229]:
# Timestamps filters
try:
    filters = [("date", "==", pd.Timestamp("2025-01-02"))]
    df_filtered = dl.to_pandas(filters=filters)
except Exception as e:
    print(e)

df_filtered

,date,ticker,close,high,low,open,volume,dt_aware,dt_naive
0,2025-01-02,AAPL,243.850006,249.100006,241.820007,248.929993,55740700,2025-01-23 00:02:40.462365+00:00,2025-01-23 00:02:40.462847
1,2025-01-02,AAPL,243.850006,249.100006,241.820007,248.929993,55740700,2025-01-23 00:02:10.380959+00:00,2025-01-23 00:02:10.381454
2,2025-01-02,AAPL,243.850006,249.100006,241.820007,248.929993,55740700,2025-01-23 00:01:40.310864+00:00,2025-01-23 00:01:40.311290
3,2025-01-02,AAPL,243.850006,249.100006,241.820007,248.929993,55740700,2025-01-23 00:01:10.242190+00:00,2025-01-23 00:01:10.242633


In [245]:
# Python datetime filters
try:
    filters = [("date", "==", datetime.strptime("2025-01-02", "%Y-%m-%d"))]
    df_filtered = dl.to_pandas(filters=filters)
except Exception as e:
    print(e)
    
df_filtered

,date,ticker,close,high,low,open,volume,dt_aware,dt_naive
0,2025-01-02,AAPL,243.850006,249.100006,241.820007,248.929993,55740700,2025-01-23 00:02:40.462365+00:00,2025-01-23 00:02:40.462847
1,2025-01-02,AAPL,243.850006,249.100006,241.820007,248.929993,55740700,2025-01-23 00:02:10.380959+00:00,2025-01-23 00:02:10.381454
2,2025-01-02,AAPL,243.850006,249.100006,241.820007,248.929993,55740700,2025-01-23 00:01:40.310864+00:00,2025-01-23 00:01:40.311290
3,2025-01-02,AAPL,243.850006,249.100006,241.820007,248.929993,55740700,2025-01-23 00:01:10.242190+00:00,2025-01-23 00:01:10.242633


## let's try to filter on the dt_aware column

In [251]:
# exact time to filter: 2025-01-23 00:02:40.462365+00:00
date_str = "2025-01-23 00:02:40.462365+00:00"
try:
    filters = [("date", "==", date_str)]
    df_filtered = dl.to_pandas(filters=filters)
except Exception as e:
    print(e)

Function 'equal' has no kernel matching input types (timestamp[us], string)


In [259]:
try:
    filters = [("dt_aware", "==", pd.Timestamp(date_str))]
    df_filtered = dl.to_pandas(filters=filters)
except Exception as e:
    print(e)
df_filtered

,date,ticker,close,high,low,open,volume,dt_aware,dt_naive
0,2025-01-02,AAPL,243.850006,249.100006,241.820007,248.929993,55740700,2025-01-23 00:02:40.462365+00:00,2025-01-23 00:02:40.462847
1,2025-01-03,AAPL,243.360001,244.179993,241.889999,243.360001,40244100,2025-01-23 00:02:40.462365+00:00,2025-01-23 00:02:40.462847


In [263]:
try:
    filters = [("dt_naive", "==", pd.Timestamp(date_str))]
    df_filtered = dl.to_pandas(filters=filters)
except Exception as e:
    print(e)

Cannot compare timestamp with timezone to timestamp without timezone, got: timestamp[us, tz=UTC] and timestamp[us]


## The deltalake schema and the data type in the filters are crutial to filter the data appropraitely

### What if I don't remember the schema?

In [347]:
df = dl.to_pandas()

In [337]:
for dt_column in ["date", "dt_aware", "dt_naive"]:
    print(f"------- {dt_column} -------")
    if pd.api.types.is_datetime64_any_dtype(df[dt_column]):
        print("This is any datetime64")
        
    if pd.api.types.is_datetime64_dtype(df[dt_column]):
        print("This is a naive datetime64")
        
    if pd.api.types.is_datetime64tz_dtype(df[dt_column]):
        print("This is a datetime64 timezone aware")        


------- date -------
This is any datetime64
This is a naive datetime64
------- dt_aware -------
This is any datetime64
This is a datetime64 timezone aware
------- dt_naive -------
This is any datetime64
This is a naive datetime64


/tmp/ipykernel_228/2765821492.py:9: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df[dt_column]):


In [339]:
for dt_column in ["date", "dt_aware", "dt_naive"]:
    print(f"------- {dt_column} -------")        
    if pd.api.types.is_datetime64_dtype(df[dt_column]):
        print("This is a naive datetime64")
        
    if pd.api.types.is_datetime64tz_dtype(df[dt_column]):
        print("This is a datetime64 timezone aware")

------- date -------
This is a naive datetime64
------- dt_aware -------
This is a datetime64 timezone aware
------- dt_naive -------
This is a naive datetime64


/tmp/ipykernel_228/2319325825.py:6: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  if pd.api.types.is_datetime64tz_dtype(df[dt_column]):


In [341]:
col_name = "dt_aware"
date_str = "2025-01-23 00:02:40.462365+00:00"

In [361]:
for dt_column in ["date", "dt_aware", "dt_naive"]:
    
    print(f"------- {dt_column} -------")
    
    if pd.api.types.is_datetime64_dtype(df[dt_column]):
        print("This is a naive datetime64")
        filter_value = pd.Timestamp(date_str).tz_localize(None)
        
    elif pd.api.types.is_datetime64tz_dtype(df[dt_column]):
        print("This is a datetime64 timezone aware")
        filter_value = pd.Timestamp(date_str)
    
    else:
        raise ValueError("Date time format not supported")
    
    try:
        filters = [(dt_column, "==", filter_value)]
        df_filtered = dl.to_pandas(filters=filters)
    except Exception as e:
        print(e)
    print(df_filtered)

------- date -------
This is a naive datetime64
Empty DataFrame
Columns: [date, ticker, close, high, low, open, volume, dt_aware, dt_naive]
Index: []
------- dt_aware -------
This is a datetime64 timezone aware
        date ticker       close        high         low        open    volume  \
0 2025-01-02   AAPL  243.850006  249.100006  241.820007  248.929993  55740700   
1 2025-01-03   AAPL  243.360001  244.179993  241.889999  243.360001  40244100   

                          dt_aware                   dt_naive  
0 2025-01-23 00:02:40.462365+00:00 2025-01-23 00:02:40.462847  
1 2025-01-23 00:02:40.462365+00:00 2025-01-23 00:02:40.462847  
------- dt_naive -------
This is a naive datetime64
Empty DataFrame
Columns: [date, ticker, close, high, low, open, volume, dt_aware, dt_naive]
Index: []


/tmp/ipykernel_228/759496241.py:9: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  elif pd.api.types.is_datetime64tz_dtype(df[dt_column]):


In [365]:
for dt_column in ["date", "dt_aware", "dt_naive"]:
    
    print(f"------- {dt_column} -------")
    
    if pd.api.types.is_datetime64_dtype(df[dt_column]):
        print("This is a naive datetime64")
        filter_value = pd.Timestamp(date_str).tz_localize(None)
        
    elif pd.api.types.is_datetime64tz_dtype(df[dt_column]):
        print("This is a datetime64 timezone aware")
        filter_value = pd.Timestamp(date_str)
    
    else:
        raise ValueError("Date time format not supported")
    
    try:
        filters = [(dt_column, ">=", filter_value)]
        df_filtered = dl.to_pandas(filters=filters)
    except Exception as e:
        print(e)
    print(df_filtered)

------- date -------
This is a naive datetime64
Empty DataFrame
Columns: [date, ticker, close, high, low, open, volume, dt_aware, dt_naive]
Index: []
------- dt_aware -------
This is a datetime64 timezone aware
        date ticker       close        high         low        open    volume  \
0 2025-01-02   AAPL  243.850006  249.100006  241.820007  248.929993  55740700   
1 2025-01-03   AAPL  243.360001  244.179993  241.889999  243.360001  40244100   

                          dt_aware                   dt_naive  
0 2025-01-23 00:02:40.462365+00:00 2025-01-23 00:02:40.462847  
1 2025-01-23 00:02:40.462365+00:00 2025-01-23 00:02:40.462847  
------- dt_naive -------
This is a naive datetime64
        date ticker       close        high         low        open    volume  \
0 2025-01-02   AAPL  243.850006  249.100006  241.820007  248.929993  55740700   
1 2025-01-03   AAPL  243.360001  244.179993  241.889999  243.360001  40244100   

                          dt_aware                   dt_n

/tmp/ipykernel_228/4113418037.py:9: DeprecationWarning: is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.
  elif pd.api.types.is_datetime64tz_dtype(df[dt_column]):
